<a href="https://colab.research.google.com/github/tathianamb/pdf_assistant/blob/gemini_v3/chatpdf_gemini_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U --quiet langchain-google-genai langchain faiss-cpu pypdf sentence-transformers PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.0 MB/s eta 0:00:00


In [2]:
%pip install -U --quiet scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 23.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [3]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import ScaNN
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from google.colab import userdata
import google.generativeai as genai


api_key = userdata.get('GOOGLE_API_KEY')
if not api_key:
    raise ValueError("Missing GOOGLE_API_KEY environment variable")

genai.configure(api_key=api_key)

os.environ["GOOGLE_API_KEY"] = api_key



In [4]:
class pdfQA:

  def __init__(self, model_path="models/embedding-001"):
    self.embeddings = GoogleGenerativeAIEmbeddings(model=model_path)
    self.chain = self._get_conversational_chain()

  def _get_pdf_text(self, pdf_docs):
      """Extracts text from a list of PDF documents."""
      text = ""
      for pdf in pdf_docs:
          pdf_reader = PdfReader(pdf)
          for page in pdf_reader.pages:
              text += page.extract_text()
      return text


  def _get_text_chunks(self, text):
      """Splits text into chunks for embedding."""
      text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
      chunks = text_splitter.split_text(text)
      return chunks


  def _create_vector_store(self, text_chunks):
      """Creates a SCANN vector store from text chunks and embeddings."""
      embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
      db = ScaNN.from_texts(text_chunks, embeddings)
      db.save_local("scann_index")


  def _get_conversational_chain(self):
      """Defines the question-answering chain using a prompt template and model."""
      prompt_template = """
      Answer the question with full context details. If you don't know the answer say "The context doesn't tell you about".\n\n
      Context:\n {context}?\n
      Question: \n{question}\n

      Answer:
      """

      model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

      prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
      chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

      return chain


  def _answer_user_question(self, user_question):
      """Answers the user's question using the conversational chain and embeddings."""
      embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

      new_db = ScaNN.load_local("scann_index", embeddings, allow_dangerous_deserialization=True)

      docs = new_db.similarity_search(user_question)

      chain = self.chain

      response = chain(
          {"input_documents": docs, "question": user_question}, return_only_outputs=True
      )

      return response["output_text"]


  def main(self):
      """Handles user interaction, PDF processing, and question answering."""
      # Consider using Colab file uploader (e.g., from google.colab import files)

      print("Chat with PDF using Gemini")

      # Load vector store or create it if it doesn't exist
      if not os.path.exists("scann_index"):
          print("Vector store not found. Processing PDFs to create it...")
          pdf_docs = input("Enter paths to your PDF files separated by commas (,) or 'upload' to upload from local machine: ")
          if pdf_docs == "upload":
              # Implement upload functionality using Colab's file uploader
              pass  # Replace with upload logic
          else:
              pdf_docs = pdf_docs.split(",")

          raw_text = self._get_pdf_text(pdf_docs)
          text_chunks = self._get_text_chunks(raw_text)
          self._create_vector_store(text_chunks)
          print("Vector store created.")

      print("Ask a Question from the PDF Files (or 'quit')\n")
      while True:
          user_question = input("User: ")
          if user_question.lower() == 'quit':
              break
          answer = self._answer_user_question(user_question)
          # You can optionally display the answer here (if not printed earlier)
          print(f"\nGemini: \n{answer}\n\n")


In [5]:
if __name__ == "__main__":
    qa_system = pdfQA()
    qa_system.main()

Chat with PDF using Gemini
Vector store not found. Processing PDFs to create it...
Enter paths to your PDF files separated by commas (,) or 'upload' to upload from local machine: /content/An_Extensive_Comparison_of_Linear_Models__Machine_Learning_approaches_and_combination_methods_for_Wind_Speed_Forecasting.pdf
Vector store created.
Ask a Question from the PDF Files (or 'quit')

User: Explain to me what a global model is and what a local model is


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Gemini: 
The context does not mention anything about global or local models, so I cannot answer this question from the provided context.


User: quit
